## LSTM (long short-term memory)长短期记忆神经网络

+ 长短期记忆神经网络和GRU很类似，只是门的类型更多，而且结构比GRU更加复杂一些

LSTM中包含以下单元：
* 输入门(input gate)
* 输出门(output gate)
* 遗忘门(forget gate)
* 与隐藏状态形状相同的记忆细胞

In [1]:
import sys
sys.path.append('../')

In [5]:
import gluonbook as gb
import mxnet as mx
from mxnet import init,autograd,gluon,nd
from mxnet.gluon import data as gdata,loss as gloss,rnn,nn

## 读取数据

In [6]:
(corpus_indices, char_to_idx, idx_to_char,
vocab_size) = gb.load_data_jay_lyrics()

In [7]:
ctx = gb.try_gpu()

## 从零开始实现LSTM

- 定义参数

In [26]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01,shape=shape,ctx=ctx)
    def _three():
        return (_one((num_inputs,num_hiddens)),_one((num_hiddens,num_hiddens)),nd.zeros(shape=(1,num_hiddens),ctx=ctx))
    
    #下面定义模型参数
    #输入门
    W_xi,W_hi,bi = _three()
    #遗忘门
    W_xf,W_hf,bf = _three()
    #输出门
    W_xo,W_ho,bo = _three()
    
    #候选记忆细胞参数
    W_xc,W_hc,bc = _three()
    
    #输出层参数
    W_hq,bq = _one((num_hiddens,num_outputs)),nd.zeros(shape = (1,num_outputs),ctx=ctx)
    
    #申请求导的梯度
    params = [W_xi,W_hi,bi,W_xf,W_hf,bf,W_xo,W_ho,bo,W_xc,W_hc,bc,W_hq,bq] 
    for param in params:
        param.attach_grad()
        
    return params

## 定义初始的隐藏层状态,以及记忆细胞状态

In [27]:
def init_lstm_state(batch_size,num_hiddens,ctx):
    return (nd.zeros(shape=(batch_size,num_hiddens),ctx=ctx),
             nd.zeros(shape=(batch_size,num_hiddens),ctx=ctx))

## 定义模型运算

In [40]:
def lstm(inputs,state,params):
    #获取参数
    W_xi,W_hi,bi,W_xf,W_hf,bf,W_xo,W_ho,bo,W_xc,W_hc,bc,W_hq,bq = params
    outputs = []
    H,C = state
    
    for X in inputs:
        #计算遗忘门、输出门、以及输入门的值
        It = nd.sigmoid(nd.dot(X,W_xi)+nd.dot(H,W_hi)+bi)
        Ft = nd.sigmoid(nd.dot(X,W_xf)+nd.dot(H,W_hf)+bf)
        Ot = nd.sigmoid(nd.dot(X,W_xo)+nd.dot(H,W_ho)+bo)
        
        #计算隐藏记忆细胞的状态
        C_temp = nd.tanh(nd.dot(X,W_xc)+nd.dot(H,W_hc)+bc)
        #计算记忆细胞的状态
        C = Ft*C+It*C_temp
        #计算隐藏层的输出
        #??怎么感觉并没有变化
        #H = Ot*nd.tanh(C)
        H = Ot*C
        #计算输出
        Y = nd.dot(H,W_hq)+bq
        outputs.append(Y)
        
    return outputs,(H,C)

In [41]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [42]:
gb.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
vocab_size, ctx, corpus_indices, idx_to_char,
char_to_idx, False, num_epochs, num_steps, lr,
clipping_theta, batch_size, pred_period, pred_len,
prefixes)

epoch 40, perplexity 208.989478, time 1.27 sec
 - 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
 - 不分开 我不的我的 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的
epoch 80, perplexity 59.714836, time 1.33 sec
 - 分开 我想你你的你 我想要你你 我不要这样 我不不你 我不了这节 我不不觉 我不了这节 我不好觉 我不了
 - 不分开 我想你你的你 我想要你你 我不要我 我不不我 我不不你 我不不这 我不不 我不 我不 我不 我不 
epoch 120, perplexity 8.312130, time 1.30 sec
 - 分开 我想要你的微笑 想想 你你再的玩笑 就通 你想再久了吧? 我想你你想很很  想穿你的你笑就就想 怎
 - 不分开 我想要你的微笑每天想想想能到 我能道这天很美但住乡的你更美美 就想的伊坦 我不多这汉汉你 我想你你
epoch 160, perplexity 2.999561, time 1.39 sec
 - 分开 我想了你你 单着银一不不 景色入秋 我天黄沙节奏 我该好好生活 不知不觉 你已经离开我 不知不觉 
 - 不分开 我已经 你你我 我想开声样牵 我不不依主舍 连隔壁我居都猜到我我想的声受受河河 祭司的神女 让要开


## 使用gluon接口实现

In [39]:
lstm_cell = rnn.LSTM(num_hiddens)
model = gb.RNNModel(lstm_cell,vocab_size)
gb.train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
corpus_indices, idx_to_char, char_to_idx,
num_epochs, num_steps, lr, clipping_theta,
batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 223.303183, time 0.11 sec
 - 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
 - 不分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
epoch 80, perplexity 64.643018, time 0.11 sec
 - 分开 我想你你的你 我有你你 我不要这我 我不要这 我不要这 我不要这 我不要这 我不好这 我不好觉 我
 - 不分开 我想你你的你 我想你你的你 我想你你 我不要这我 我不要这 我不要这 我不要这 我不要这 我不好觉
epoch 120, perplexity 13.781476, time 0.12 sec
 - 分开 我想想你 我我要要 你你在 是怎我 说你怎 是你怎么的你说 说你了么我说说 说说球 我想我要难头 
 - 不分开你的让面 想想要你想经单单 想想 你想我想我 说样 我想很久久吧  说  你给很了吧吧 像散  又给
epoch 160, perplexity 3.605985, time 0.11 sec
 - 分开 我想带你 我不多难熬我 没你你烦我有一场恼剧 我感那这了坦美不家 不懂你的你 让我的美 你经的美样
 - 不分开你 经经的假女 我要要你我 我有 我不要 我想了这生活 不知不觉 你已经离开我 不知不觉 我跟了这节
epoch 200, perplexity 1.653556, time 0.11 sec
 - 分开 别是我 是是是枪手 巫师 他念念 有有的 在长长 不上变明的片墙 干什么 干什么 已行病一夫招开 
 - 不分开走走封面 想要要这不坦 却你都童话打我 我想 你原你堡每每你 不不开 一颗我抬起头 说话去么医药箱说
epoch 240, perplexity 1.244419, time 0.12 sec
 - 分开 我想悔的你你 每常依不不气气气 就么一九 三子我 别经我  没有你在我有多 恼多就我 你这球球 我
 - 不分开觉 后作你 开简我 我不大声宣对 对你依依不舍 连隔壁邻居都猜到我现在的感受 河边的风 在吹着头发飘
epoch 280, perplexity 1.118267, time 0.11 se

- 既然候选记忆细胞已通过使⽤ tanh 函数确保值域在 -1 到 1 之间，为什么隐藏状态还需再次使⽤ tanh 函数来确保输出值域在 -1 到 1 之间？

由于当tanh输出为接近-1或1时梯度很小，更新很慢，所以再进行一次映射，可以增加梯度，从而加快收敛速度